# Setup Dependencies

In [5]:
%%capture --no-stderr
%pip install --upgrade --quiet psycopg2 langchain langchain-community langchain-openai faiss-cpu langgraph python-dotenv
%pip install --extra-index-url https://europe-west3-python.pkg.dev/bionic-unity-294411/radicalbit-python-api/simple radicalbit-client==3.4.4

In [6]:
def beautify_gpt_response(response):
    return response[list(response.keys())[0]]['messages'][0].pretty_repr()

# Setup Database

In [7]:
from langchain_community.utilities import SQLDatabase
from dotenv import load_dotenv
import os

load_dotenv()

db_host = os.environ.get("DATABASE_HOST")
db_port = os.environ.get("DATABASE_PORT")
db_name = os.environ.get("DATABASE_NAME")
db_user = os.environ.get("DATABASE_USER")
db_password = os.environ.get("DATABASE_PASSWORD")

db = SQLDatabase.from_uri(
    f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}",
)
print(f"SQL dialect: {db.dialect}")
print(f"List of usable tables: {db.get_usable_table_names()}")

SQL dialect: postgresql
List of usable tables: ['fleet_model', 'vehicle', 'vehicle_position', 'vehicle_stats']


# Create Agent

In [8]:
import os
api_key = os.environ.get("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", api_key=api_key)

In [9]:
import json
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

for t in tools:
    print(json.dumps(t.to_json(), indent=4))

{
    "lc": 1,
    "type": "not_implemented",
    "id": [
        "langchain_community",
        "tools",
        "sql_database",
        "tool",
        "QuerySQLDataBaseTool"
    ],
    "repr": "QuerySQLDataBaseTool(description=\"Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.\", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x123debe30>)",
    "name": "sql_db_query"
}
{
    "lc": 1,
    "type": "not_implemented",
    "id": [
        "langchain_community",
        "tools",
        "sql_database",
        "tool",
        "InfoSQLDatabaseTool"
    ],
    "repr": "InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is

In [10]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer.
Unless the user specifies that want to retrieve a specific number of examples they wish to obtain, always limit your query to at most 5 results.
If the user ask for all the examples do not limit the query to 5 result.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
ALWAYS use double quote to identify table name.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

In [11]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, state_modifier=system_message)

# Test human language query

In [12]:
for s in agent_executor.stream(
        {"messages": [HumanMessage(content="list truck models of MAN")]}
):
    print(beautify_gpt_response(s))

================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_RLA3O7tNdDPuiB7B8VPbW7w8)
 Call ID: call_RLA3O7tNdDPuiB7B8VPbW7w8
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

fleet_model, vehicle, vehicle_position, vehicle_stats
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_olvi4o24OZBN1w4PhglI4gBb)
 Call ID: call_olvi4o24OZBN1w4PhglI4gBb
  Args:
    table_names: fleet_model
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE fleet_model (
	id SERIAL NOT NULL, 
	name VARCHAR(255) NOT NULL, 
	manufacturer VARCHAR(255) NOT NULL, 
	year INTEGER NOT NULL, 
	CONSTRAINT fleet_model_pkey PRIMARY KEY (id)
)

/*
3 rows from fleet_model table:
id	name	manufacturer	year
1	FH	Volvo	2012
2	Actros	Mercedes-Benz	2011
3	R Series	Scania	2004
*/
===

In [9]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="list truck models of MANN")]}
):
    print(beautify_gpt_response(s))

================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_BPL9liQokCmgfwl39wua1MwB)
 Call ID: call_BPL9liQokCmgfwl39wua1MwB
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

fleet_model, vehicle, vehicle_position, vehicle_stats
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_G6av9IfIAGSSozbhjFkZuj4K)
 Call ID: call_G6av9IfIAGSSozbhjFkZuj4K
  Args:
    table_names: vehicle
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE vehicle (
	plate VARCHAR(255) NOT NULL, 
	model_id INTEGER, 
	color VARCHAR(255) NOT NULL, 
	CONSTRAINT vehicle_pkey PRIMARY KEY (plate), 
	CONSTRAINT fk_model FOREIGN KEY(model_id) REFERENCES fleet_model (id)
)

/*
3 rows from vehicle table:
plate	model_id	color
DC550WF	22	Black
BD458MM	22	Black
CE303DR	3	Black
*

# Embrace the typos with Embeddings

In [ ]:
from langchain.evaluation import load_evaluator
from langchain_openai import OpenAIEmbeddings

evaluator = load_evaluator(evaluator="embedding_distance",
                           embeddings=OpenAIEmbeddings())

print(evaluator.evaluate_strings(prediction="Amsterdam", reference="coffeeshop"))
print(evaluator.evaluate_strings(prediction="Tokyo", reference="coffeeshop"))

In [13]:
import ast
import re

def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))

names = query_as_list(db, "SELECT name FROM fleet_model")
manufacturers = query_as_list(db, "SELECT manufacturer FROM fleet_model")
colors = query_as_list(db, "SELECT color FROM vehicle")
zone = query_as_list(db, "SELECT zone FROM vehicle_position")
city = query_as_list(db, "SELECT city FROM vehicle_position")
manufacturers[:5]

['Renault', 'Scania', 'Volvo', 'MAN', 'DAF']

In [14]:
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vector_db = FAISS.from_texts(names + manufacturers + colors + zone + city, OpenAIEmbeddings())
retriever = vector_db.as_retriever(search_kwargs={"k": 5})
description = """Use to look up values to filter on. Input is an approximate spelling of the proper noun, output is \
valid proper nouns. Use the noun most similar to the search."""
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)
print(retriever_tool.invoke("MANN"))

MAN

Maenza

Messina

Messina

LF


In [12]:
system = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
If the user ask for all the examples do not limit the query to 5 result.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

You have access to the following tables: {table_names}

If you need to filter on a proper noun, you must ALWAYS first look up the filter value using the "search_proper_nouns" tool!
Do not try to guess at the proper name - use this function to find similar ones.""".format(
    table_names=db.get_usable_table_names()
)

system_message = SystemMessage(content=system)

tools.append(retriever_tool)

agent = create_react_agent(llm, tools, state_modifier=system_message)

for t in tools:
    print(json.dumps(t.to_json(), indent=4))

{
    "lc": 1,
    "type": "not_implemented",
    "id": [
        "langchain_community",
        "tools",
        "sql_database",
        "tool",
        "QuerySQLDataBaseTool"
    ],
    "repr": "QuerySQLDataBaseTool(description=\"Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.\", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x105399a60>)",
    "name": "sql_db_query"
}
{
    "lc": 1,
    "type": "not_implemented",
    "id": [
        "langchain_community",
        "tools",
        "sql_database",
        "tool",
        "InfoSQLDatabaseTool"
    ],
    "repr": "InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is

In [13]:
for s in agent.stream(
        {"messages": [HumanMessage(content="list truck models of MANN")]}
):
    print(beautify_gpt_response(s))

================================== Ai Message ==================================
Tool Calls:
  search_proper_nouns (call_WOKGEjFYEBC1UxvqbMCbH7A2)
 Call ID: call_WOKGEjFYEBC1UxvqbMCbH7A2
  Args:
    query: MANN
================================= Tool Message =================================
Name: search_proper_nouns

MAN

Maenza

Messina

Messina

LF
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_D5lWI4z5L7xmh0yAXyTfUBVD)
 Call ID: call_D5lWI4z5L7xmh0yAXyTfUBVD
  Args:
    table_names: fleet_model
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE fleet_model (
	id SERIAL NOT NULL, 
	name VARCHAR(255) NOT NULL, 
	manufacturer VARCHAR(255) NOT NULL, 
	year INTEGER NOT NULL, 
	CONSTRAINT fleet_model_pkey PRIMARY KEY (id)
)

/*
3 rows from fleet_model table:
id	name	manufacturer	year
1	FH	Volvo	2012
2	Actros	Mercedes-Benz	2011
3	R Series	Scania	2004
*/
=====

# Enhance response with Radicalbit RAG

In [14]:
for s in agent.stream(
    {"messages": [HumanMessage(content="How many vehicle manufactured by Scana exist?")]}
):
    print(beautify_gpt_response(s))

================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_4CDTty5OJpOAjSToJ8JG5FGv)
 Call ID: call_4CDTty5OJpOAjSToJ8JG5FGv
  Args:
    table_names: fleet_model
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE fleet_model (
	id SERIAL NOT NULL, 
	name VARCHAR(255) NOT NULL, 
	manufacturer VARCHAR(255) NOT NULL, 
	year INTEGER NOT NULL, 
	CONSTRAINT fleet_model_pkey PRIMARY KEY (id)
)

/*
3 rows from fleet_model table:
id	name	manufacturer	year
1	FH	Volvo	2012
2	Actros	Mercedes-Benz	2011
3	R Series	Scania	2004
*/
================================== Ai Message ==================================
Tool Calls:
  search_proper_nouns (call_ZpE2gh9AeQ8k733Vc3IxHEra)
 Call ID: call_ZpE2gh9AeQ8k733Vc3IxHEra
  Args:
    query: Scana
================================= Tool Message =================================
Name: search_proper_nouns

Scandicci

Scania

Scarmagno

Scilla



In [165]:
from radicalbit_client.rag import RadicalbitRagClient
from dotenv import load_dotenv
import os

load_dotenv()

client_id = os.environ.get("RB_CLIENT_ID")
client_secret = os.environ.get("RB_CLIENT_SECRET")

# Create and instance of the Radicalbit RAG Client
radicalbit_rag_client = RadicalbitRagClient(
    host="platform.radicalbit.ai",
    port ="443",
    client_id=client_id,
    client_secret=client_secret,
    tenant_name="bitrockcodemotion2024"
)

agent_answer = s["agent"]["messages"][-1].content

payload = radicalbit_rag_client.invoke(prompt=agent_answer)
print(payload['answer'])

Based on the provided context, here is a description of the Scania truck models mentioned:

1. **Scania S Series (2016)**
   - **Description**: The S Series is Scania's premium range designed for long-haul operations. It features a flat floor cab for maximum space, advanced driver support systems, and highly efficient powertrains. Known for its luxurious interior and excellent fuel economy.

2. **Scania R Series (2004)**
   - **Description**: The R Series is versatile, suitable for various applications including long-haul and construction. It's known for robust build quality, fuel efficiency, and driver comfort. Offers a range of powertrains, including options for alternative fuels, and is reputed for reliability and strong residual values.

3. **Scania P Series (2004)**
   - **Description**: The P Series is ideal for urban and regional distribution, as well as some construction applications. It features a lower cab for easy entry and exit, making it perfect for multi-drop operations. 